# Methods Scratchpad

A notebook for the trialling of analytical methods for the detection of layer number from graphene diffraction patterns, before those methods are incorporated into the main project.

## Imports and Setup

In [7]:
%matplotlib qt
import os
import sys
import time

import numpy as np
import scipy.optimize
import sklearn.cluster
import matplotlib.pyplot as plt
import skimage.feature
import skimage.filters
import skimage.morphology

import hyperspy.api as hs

sys.path.append("../")
import windows
import methods

In [8]:
DATA_DIRECTORY = "../data"
data_filenames = []

# Make a list of filenames
for dataset_directory in os.listdir(DATA_DIRECTORY):
    dataset_directory_path = os.path.join(DATA_DIRECTORY, dataset_directory)
    data_filenames.append({
        "dataset_name": dataset_directory,
        "dataset_file_list": [
            os.path.join(dataset_directory_path, filename) 
            for filename in os.listdir(dataset_directory_path) 
            if filename[-4:] == ".dm3" or filename[-4:] == ".emd"
        ]
    })

## Initial peakfinding

In [9]:
s = hs.load(data_filenames[1]["dataset_file_list"][7])

In [10]:
peaks_list = skimage.feature.peak_local_max(s.data[:], min_distance=100, threshold_abs=100, exclude_border=True, num_peaks=12)
peaks_list_new = methods.Position.from_list(peaks_list)

## Approximate Spot Size Estimation

In [11]:
def gaussian(pos, xo, yo, sigma, amplitude, offset):
    """Returns a 2D gaussian function flattened to a 1D array."""  
    g = offset + amplitude*np.exp( - (((pos[0]-xo)**2)/(2*sigma**2) + ((pos[1]-yo)**2)/(2*sigma**2)))
    return g.ravel()

def get_gaussian_fwhm(img):
    x = np.linspace(0, img.shape[1], img.shape[1])
    y = np.linspace(0, img.shape[0], img.shape[0])
    x, y = np.meshgrid(x, y)
    #Parameters: xpos, ypos, sigma, amp, baseline
    initial_guess = (img.shape[1]/2,img.shape[0]/2,10,1,0)
    # subtract background and rescale image into [0,1], with floor clipping
    bg = np.percentile(img,5)
    img_scaled = np.clip((img - bg) / (img.max() - bg),0,1)
    popt, pcov = scipy.optimize.curve_fit(gaussian, 
                                          (x, y), 
                                          img_scaled.ravel(), 
                                          p0=initial_guess, 
                                          bounds=((img.shape[1]*0.4, img.shape[0]*0.4, 1, 0.5, -0.1), 
                                                  (img.shape[1]*0.6, img.shape[0]*0.6, img.shape[0]/2, 1.5, 0.5)))
    xcenter, ycenter, sigma, amp, offset = popt[0], popt[1], popt[2], popt[3], popt[4]
    FWHM = np.abs(4*sigma*np.sqrt(-0.5*np.log(0.5)))
    return FWHM

Select a single peak to fit to a gaussian, and then assume all peaks have the same FWHM

In [12]:
test_peak_coord = peaks_list[0]

In [13]:
gaussian_fit_sub_window_half_size = (100, 100)
gaussian_fit_sub_window = s.data[test_peak_coord[0] - gaussian_fit_sub_window_half_size[0] : \
                                 test_peak_coord[0] + gaussian_fit_sub_window_half_size[0],
                                 test_peak_coord[1] - gaussian_fit_sub_window_half_size[0] : \
                                 test_peak_coord[1] + gaussian_fit_sub_window_half_size[0]]
spot_fwhm = get_gaussian_fwhm(gaussian_fit_sub_window)

Define sizes of selected areas etc. relative to the FWHM

In [14]:
selected_radius = spot_fwhm * 2
inner_background_radius = spot_fwhm * 5
outer_background_radius = spot_fwhm * 6

Plot it for debugging

In [15]:
fig, ax = plt.subplots()
ax.imshow(gaussian_fit_sub_window, cmap="Greys")
circle = plt.Circle(gaussian_fit_sub_window_half_size, spot_fwhm, color='r', fill=False)
circle_sa = plt.Circle(gaussian_fit_sub_window_half_size, selected_radius, color='g', fill=False)
circle_bg_inner = plt.Circle(gaussian_fit_sub_window_half_size, inner_background_radius, color='g', fill=False)
circle_bg_outer = plt.Circle(gaussian_fit_sub_window_half_size, outer_background_radius, color='g', fill=False)
ax.add_patch(circle)
ax.add_patch(circle_sa)
ax.add_patch(circle_bg_inner)
ax.add_patch(circle_bg_outer)
plt.show()

## Intensity calculation

Filters appear to be un-necessary...

In [16]:
s_dilated = skimage.filters.gaussian(s.data, sigma=spot_fwhm)
# mean_filter_footprint = skimage.morphology.disk(selected_radius//2)
# s_dilated = skimage.filters.rank.mean(s.data, mean_filter_footprint)
# s_dilated = s.data

In [17]:
peak_intensities = []
image_window = windows.CircularWindow(selected_radius, inner_background_radius, outer_background_radius, s_dilated)
for peak_point in peaks_list:
    peak_intensities.append(image_window.get_integrated_intensity(*peak_point))

Plot a single peak's apertures, for debugging.

In [18]:
debug_peak_number = 9
print("Shown peak intensity:", peak_intensities[debug_peak_number])

fig, axis = plt.subplots()
masks, sub_image = image_window.show_masks(*(peaks_list[debug_peak_number]))
axis.imshow(sub_image, cmap="gray")
axis.imshow(masks, cmap="jet", alpha=0.5)
plt.show()

Shown peak intensity: 24259.479638930938


## Refine Spot peak-finding

Refine the estimates classification of spots with the image by using the knowledge that they should all be centered around a central point.

In [13]:
central_spot_estimate = np.mean(peaks_list, axis=0)

Plot the location of the mean-central spot.

In [14]:
fig, ax = plt.subplots()
# ax.imshow(s_dilated, vmin=0, vmax=250)
ax.imshow(s.data, vmin=0, vmax=200)
max_intensity = max(peak_intensities)
for peak_point, peak_intensity in zip(peaks_list, peak_intensities):
    color = 'g'
    if 0.4 < peak_intensity / max_intensity < 0.65:
        color = 'r'
    circle = plt.Circle(peak_point[::-1], outer_background_radius*1.1, color=color, fill=False)
    ax.add_patch(circle)
central_spot_patch = plt.Circle(central_spot_estimate[::-1], outer_background_radius, color='r', fill=True)
ax.add_patch(central_spot_patch)
plt.show()

Now use this rough estimate to try to refine the central point

In [15]:
distances_to_central_spot = np.linalg.norm([peak_pos - central_spot_estimate for peak_pos in peaks_list], axis=1)
distances_to_central_spot = np.array(list(zip(peaks_list, distances_to_central_spot)))

In [16]:
distances_to_central_spot[:] = distances_to_central_spot[distances_to_central_spot[:,1].argsort()]

In [17]:
radial_distance_steps = np.diff(distances_to_central_spot[:, 1])

In [18]:
radial_distance_steps_median = np.median(radial_distance_steps)

In [19]:
radial_distance_steps_normalised = radial_distance_steps / radial_distance_steps_median

In [20]:
big_step_indices = []
for index, normalised_radial_step in enumerate(radial_distance_steps_normalised):
    if normalised_radial_step > 5:
        big_step_indices.append(index+1)
print(big_step_indices)

[6]


In [21]:
radially_ordered_peak_positions = distances_to_central_spot[:, 0][:]
radial_position_clusters = []
previous_step_index = 0
for big_step_index in big_step_indices:
    radial_position_clusters.append(radially_ordered_peak_positions[previous_step_index: big_step_index])
    previous_step_index = big_step_index
radial_position_clusters.append(radially_ordered_peak_positions[big_step_index:])
print(radial_position_clusters)

[array([array([2554, 1862]), array([1493, 2150]), array([2413, 2396]),
       array([1634, 1616]), array([2164, 1472]), array([1883, 2541])],
      dtype=object), array([array([1101, 1759]), array([2947, 2254]), array([1350, 2686]),
       array([2698, 1326]), array([2273, 2934]), array([1775, 1078])],
      dtype=object)]


In [ ]:
def positions_equal(pos1, pos2):
    return all([pos1_coord==pos2_coord for pos1_coord, pos2_coord in zip(pos1, pos2)])

In [25]:
radial_position_index_clusters = []
for radial_position_cluster in radial_position_clusters:
    cluster_indices = []
    for position in radial_position_cluster:
        for index, peak_pos in enumerate(peaks_list):
            if all([peak_pos_coord == position_coord for peak_pos_coord, position_coord in zip(peak_pos, position)]):
                cluster_indices.append(index)
                continue
    radial_position_index_clusters.append(cluster_indices)
total_indices = 0
for cluster in radial_position_index_clusters:
    total_indices += len(cluster)
print(len(peaks_list), total_indices)

12 12


Plot the whole diffraction pattern with identified peaks, for debugging.

In [22]:
fig, ax = plt.subplots()
# ax.imshow(s_dilated, vmin=0, vmax=250)
ax.imshow(s.data, vmin=0, vmax=200)
max_intensity = max(peak_intensities)
for peak_point, peak_intensity in zip(peaks_list, peak_intensities):
    color = 'g'
    if peak_point in radial_position_clusters[0]:
        color='r'
    circle = plt.Circle(peak_point[::-1], outer_background_radius*1.1, color=color, fill=False)
    ax.add_patch(circle)
central_spot_patch = plt.Circle(central_spot_estimate[::-1], outer_background_radius, color='r', fill=True)
ax.add_patch(central_spot_patch)
plt.show()

/tmp/ipykernel_65893/2642161558.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if peak_point in radial_position_clusters[0]:


Plot a histogram of peak intensities, for classification debugging.

In [18]:
fig, ax = plt.subplots()
ax.hist([peak_intensity / max_intensity for peak_intensity in peak_intensities], bins=10)
plt.show()